In [2]:
from pyspark.sql import SparkSession


In [3]:

spark = SparkSession.builder\
        .appName("Pyspark By Examples")\
        .getOrCreate()
print(spark)

25/03/12 11:59:34 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.243.232 instead (on interface wlp0s20f3)
25/03/12 11:59:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/12 11:59:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dataDF = [(("James","","Smith"),'1991-04-01','M',3000),
         (("Michael","Rose",""),'2000-05-19','M',4000),
         (('Robert','','Williams'),'1978-09-05','M',4000),
          (('Maria','Anne','Jones'),'1967-12-01','F',4000),
          (('Jen','Mary','Brown'),'1980-02-17','F',-1)]

In [5]:
from pyspark.sql.types import *

In [15]:
schema = StructType([
    StructField("name",StructType([
        StructField("firstname",StringType(),True),
        StructField("middlename",StringType(),True),
        StructField("lastname",StringType(),True)
    ]),True),
    StructField("dob",StringType(),True),
    StructField("gender",StringType(),True),
    StructField("salary",IntegerType(),True)
])

In [16]:
df = spark.createDataFrame(data = dataDF,schema =schema)

In [17]:
df.show()

+--------------------+----------+------+------+
|                name|       dob|gender|salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



In [18]:
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



withColumnRenamed()

In [19]:
df.withColumnRenamed("dob","DateOfBirth").show()

+--------------------+-----------+------+------+
|                name|DateOfBirth|gender|salary|
+--------------------+-----------+------+------+
|    {James, , Smith}| 1991-04-01|     M|  3000|
|   {Michael, Rose, }| 2000-05-19|     M|  4000|
|{Robert, , Williams}| 1978-09-05|     M|  4000|
|{Maria, Anne, Jones}| 1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}| 1980-02-17|     F|    -1|
+--------------------+-----------+------+------+



for multicolumn renaming()

In [20]:
df.withColumnRenamed("dob","DateOfBirth")\
    .withColumnRenamed("salary","salary_amount").printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary_amount: integer (nullable = true)



rename nested column

In [22]:
df.select(df.name.firstname.alias("fname"),
         df.name.middlename.alias("mname"),
         df.name.lastname.alias("lname"),
         df.dob,
         df.gender,
         df.salary).show()

+-------+-----+--------+----------+------+------+
|  fname|mname|   lname|       dob|gender|salary|
+-------+-----+--------+----------+------+------+
|  James|     |   Smith|1991-04-01|     M|  3000|
|Michael| Rose|        |2000-05-19|     M|  4000|
| Robert|     |Williams|1978-09-05|     M|  4000|
|  Maria| Anne|   Jones|1967-12-01|     F|  4000|
|    Jen| Mary|   Brown|1980-02-17|     F|    -1|
+-------+-----+--------+----------+------+------+



In [25]:
df.withColumn("fname",df.name.firstname)\
    .withColumn("mname",df.name.middlename)\
    .withColumn("lname",df.name.lastname)\
    .drop(df.name).show()

+----------+------+------+-------+-----+--------+
|       dob|gender|salary|  fname|mname|   lname|
+----------+------+------+-------+-----+--------+
|1991-04-01|     M|  3000|  James|     |   Smith|
|2000-05-19|     M|  4000|Michael| Rose|        |
|1978-09-05|     M|  4000| Robert|     |Williams|
|1967-12-01|     F|  4000|  Maria| Anne|   Jones|
|1980-02-17|     F|    -1|    Jen| Mary|   Brown|
+----------+------+------+-------+-----+--------+



In [26]:
df.show()

+--------------------+----------+------+------+
|                name|       dob|gender|salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



## withColummn(_)

In [70]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data,schema =columns)

In [71]:
df.show()
df.printSchema()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



Changing the datatype

In [30]:
df.withColumn("salary",df.salary.cast("Integer")).printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [31]:
df.withColumn("salary",df.salary*100).show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|300000|
|  Michael|      Rose|        |2000-05-19|     M|400000|
|   Robert|          |Williams|1978-09-05|     M|400000|
|    Maria|      Anne|   Jones|1967-12-01|     F|400000|
|      Jen|      Mary|   Brown|1980-02-17|     F|  -100|
+---------+----------+--------+----------+------+------+



In [32]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [33]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [37]:
from pyspark.sql.functions import *


### when(_)

In [38]:
from pyspark.sql.functions import when

In [39]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [40]:
df.withColumn("bonus",when(df.gender=='F',df.salary*0.15)\
             .otherwise(df.salary*0.10)).show()

+---------+----------+--------+----------+------+------+-----+
|firstname|middlename|lastname|       dob|gender|salary|bonus|
+---------+----------+--------+----------+------+------+-----+
|    James|          |   Smith|1991-04-01|     M|  3000|300.0|
|  Michael|      Rose|        |2000-05-19|     M|  4000|400.0|
|   Robert|          |Williams|1978-09-05|     M|  4000|400.0|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|600.0|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|-0.15|
+---------+----------+--------+----------+------+------+-----+



In [50]:
df.withColumn("tax_incured", when (df.salary<0,0)\
             .when(df.salary<4000,df.salary*0.5)\
             .otherwise(df.salary*0.10)).toPandas()

,firstname,middlename,lastname,dob,gender,salary,tax_incured
0,James,,Smith,1991-04-01,M,3000,1500.0
1,Michael,Rose,,2000-05-19,M,4000,400.0
2,Robert,,Williams,1978-09-05,M,4000,400.0
3,Maria,Anne,Jones,1967-12-01,F,4000,400.0
4,Jen,Mary,Brown,1980-02-17,F,-1,0.0


#### creating a Temp View

In [51]:
df.createOrReplaceTempView("EMP")

In [52]:
spark.sql("select * from EMP").show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [54]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |      emp|       true|
+---------+---------+-----------+



In [61]:
gender = df.gender

In [68]:
print(gender)

Column<'gender'>


In [72]:
# df = df.withColumn("gender",gender)

In [58]:
df = df.drop("tax_incured")

In [63]:
df.show()

+---------+----------+--------+----------+------+
|firstname|middlename|lastname|       dob|salary|
+---------+----------+--------+----------+------+
|    James|          |   Smith|1991-04-01|  3000|
|  Michael|      Rose|        |2000-05-19|  4000|
|   Robert|          |Williams|1978-09-05|  4000|
|    Maria|      Anne|   Jones|1967-12-01|  4000|
|      Jen|      Mary|   Brown|1980-02-17|    -1|
+---------+----------+--------+----------+------+



In [64]:
spark.sql("select * from EMP").show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [73]:
spark.stop()